모델
처음에는 lgbm, xgboost, catboost 세가지의 모델의 결과의 앙상블을 통해 순위를 조금씩 올렸습니다. 

최종적으로는 lgbm 단일모델로 하이퍼 파라미터를 조율하며 마무리했습니다.

그리고 다중 분류 방법 중 one verse all 기법을 이용해보고 싶었는데 좋은 자료를 찾지못해 포기한점이 아쉽습니다.
lgbm에서는 objective = 'multicalssova'을 이용하면 쉽게 사용할 수 있는데 제경우에는 overfitting이 심해져 포기했습니다.



변수 
초반에는 fiberID를 이용해 1000개의 모델을 만들어보려고 했는데 학습 데이터를 나누면 train label과 validation label이 달라 학습되지 않는 부분이 해결되지 않아 포기했습니다.
validation 없이 진행 해서 적용해보았지만 결과가 좋지 못했습니다.
하지만 이과정에서 QSO만 관측한 fiberID를 찾아낼 수 있었고 이 fiberID들의 결과를 모두 QSO로 확정한것이 미세하게 점수를 올리는데 도움된것 같습니다.


FEATURE Engineering
 파장별, 관측법 별 mean, std를 구하고 변수별로 pca를 적용했습니다. 
mean과 std에도 pca를 적용했는데 std의 pca 변수가 모델의 feature importance의 상위권에 올라져 있는 이유를 아직도 모르겠습니다.

일주일을 채 남겨두지 않고, 토론글을 통해 도메인 지식을 이용해 새로운 변수를 생성할 수 있었습니다. 



하이퍼 파라미터
하이퍼 파라미터는 베이지안 옵티마이제이션을 이용해 적절한 변수를 찾고 최종적으로 num_leaves와 max_depth, learning_rate를 조절하며 최선의 하이퍼파라미터를 찾았습니다.

type : 천체의 분류(예측 변수)

psfMag : Point spread function magnitudes 빛의 밝기

fiberMag : 광섬유를 통과하는 빛의 밝기

petroMag : 빛의 밝기를 비교하기 위한 수치

medelMag : 천체 중심으로부터 특정 거리의 밝기

fiberID : 관측에 사용된 광섬유의 구분자

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import math 

from sklearn.ensemble import IsolationForest

In [2]:
train = pd.read_csv("C:/dataset/dacon_SDSS/train.csv", index_col=0)
test = pd.read_csv("C:/dataset/dacon_SDSS/test.csv", index_col=0)
sample_submission = pd.read_csv("C:/dataset/dacon_SDSS/sample_submission.csv", index_col=0)

In [3]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,22.504317,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,21.360701,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,17.867253,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,20.433907,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,25.859229,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655


In [6]:
test.head()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,...,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
id,,,,,,,,,,,,,,,,,,,,,
199991,251,23.817399,22.508963,20.981106,18.517316,17.076079,25.053890,23.167848,21.335901,18.835858,...,22.246697,22.796239,21.195315,18.584486,17.154284,25.391534,22.499435,21.011918,18.499341,17.091474
199992,386,22.806983,21.937111,20.335770,20.000512,19.527369,22.498565,22.186000,20.618879,20.301204,...,21.729831,21.837511,20.196128,19.967204,19.683671,22.475338,21.853442,20.173169,19.796757,19.567372
199993,232,21.024250,19.235669,18.304061,17.808608,17.380113,21.205546,19.439533,18.344433,17.909690,...,20.722629,18.710223,17.611851,17.158519,16.843986,20.579314,18.653338,17.562108,17.120529,16.708748
199994,557,20.503424,20.286261,20.197204,20.162419,20.059832,20.976132,20.611498,20.567262,20.479318,...,20.329269,20.385262,20.129157,20.206574,20.212342,20.479879,20.280943,20.150499,20.206221,20.092909
199995,75,24.244851,22.668237,21.239333,19.284777,18.235939,25.681860,22.935289,21.642456,19.624926,...,22.308298,22.957496,21.285033,19.299120,18.307526,25.489360,22.857290,21.191862,19.237964,18.280368


In [9]:
sample_submission.head()

,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
id,,,,,,,,,,,,,,,,,,,
199991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Feature engineering

FiberID 별로 관측된 천체 타입의 종류가 다름을 인지하고 이에 따른 Typenum 변수 생성

특히 640? 이후로는 모든 변수가 QSO임을 확인하고 이를 제외하고 train하기로 마음먹음

In [3]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))


In [4]:
sample_submission.columns

Index(['STAR_WHITE_DWARF', 'STAR_CATY_VAR', 'STAR_BROWN_DWARF',
       'SERENDIPITY_RED', 'REDDEN_STD', 'STAR_BHB', 'GALAXY',
       'SERENDIPITY_DISTANT', 'QSO', 'SKY', 'STAR_RED_DWARF', 'ROSAT_D',
       'STAR_PN', 'SERENDIPITY_FIRST', 'STAR_CARBON', 'SPECTROPHOTO_STD',
       'STAR_SUB_DWARF', 'SERENDIPITY_MANUAL', 'SERENDIPITY_BLUE'],
      dtype='object')

In [10]:
column_number

{'STAR_WHITE_DWARF': 0,
 'STAR_CATY_VAR': 1,
 'STAR_BROWN_DWARF': 2,
 'SERENDIPITY_RED': 3,
 'REDDEN_STD': 4,
 'STAR_BHB': 5,
 'GALAXY': 6,
 'SERENDIPITY_DISTANT': 7,
 'QSO': 8,
 'SKY': 9,
 'STAR_RED_DWARF': 10,
 'ROSAT_D': 11,
 'STAR_PN': 12,
 'SERENDIPITY_FIRST': 13,
 'STAR_CARBON': 14,
 'SPECTROPHOTO_STD': 15,
 'STAR_SUB_DWARF': 16,
 'SERENDIPITY_MANUAL': 17,
 'SERENDIPITY_BLUE': 18}

In [11]:
train.head()

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type_num
id,,,,,,,,,,,,,,,,,,,,,
0,QSO,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,...,21.431636,21.478312,21.145409,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340,8
1,QSO,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,...,20.778968,20.889705,20.639812,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314,8
2,QSO,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,...,16.738784,16.688874,16.744210,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928,8
3,QSO,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,...,19.993727,19.985531,19.750917,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855,8
4,STAR_RED_DWARF,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,...,22.426929,21.673551,19.610012,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655,10


In [5]:
# Make Fiber tpye
fiber = []
label_dict = {}

for i in list(set(train['fiberID'])):
    train2 = train[train['fiberID']==i]
    #test2 = test[test['fiberID']==i]
    #idx2 = test2.index
    idx1 = train2.index
    # print(set(train.loc[idx1]['type_num']))
    if tuple(set(train.loc[idx1]['type_num'])) not in fiber:
        fiber.append(tuple(set(train.loc[idx1]['type_num'])))
    label_dict[i] = list(set(train.loc[idx1]['type_num']))
    
    
fiber_type = {}
for k in range(len(fiber)):
    fiber_type[fiber[k]] = k
    
fiber_dict = {}  
for j in list(set(train['fiberID'])):
    train2 = train[train['fiberID']==j]
    #test2 = test[test['fiberID']==i]
    idx1 = train2.index
    
    #print(set(train.loc[idx1]['type_num']))
    fiber_dict[j] = fiber_type[tuple(set(train.loc[idx1]['type_num']))]


In [6]:
label_dict[1]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18]

In [25]:
fiber[1]

(0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18)

In [8]:
label_dict

, 9, 10, 11, 13, 14, 15, 16, 18],
 381: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 18],
 382: [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 383: [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 384: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 385: [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 386: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 387: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18],
 388: [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 389: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 18],
 390: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 391: [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 392: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 18],
 393: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 394: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 18],
 395: [0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18],
 396: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11,

In [24]:
fiber_type

{(0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18): 0,
 (0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18): 1,
 (0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 18): 2,
 (0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 18): 3,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 18): 4,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 18): 5,
 (0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 18): 6,
 (0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 17, 18): 7,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18): 8,
 (0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18): 9,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 18): 10,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18): 11,
 (0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18): 12,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 17, 18): 13,
 (0, 1, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18): 14,
 (0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 15, 16, 18): 15,
 (1, 2, 3, 4, 5, 6, 7, 8, 1

In [26]:
fiber_dict

{1: 0,
 2: 1,
 3: 2,
 4: 0,
 5: 1,
 6: 0,
 7: 3,
 8: 1,
 9: 4,
 10: 0,
 11: 3,
 12: 1,
 13: 1,
 14: 2,
 15: 1,
 16: 0,
 17: 1,
 18: 1,
 19: 2,
 20: 0,
 21: 5,
 22: 1,
 23: 1,
 24: 6,
 25: 3,
 26: 3,
 27: 7,
 28: 8,
 29: 1,
 30: 1,
 31: 1,
 32: 4,
 33: 1,
 34: 1,
 35: 1,
 36: 4,
 37: 0,
 38: 0,
 39: 1,
 40: 1,
 41: 9,
 42: 7,
 43: 0,
 44: 0,
 45: 4,
 46: 0,
 47: 1,
 48: 2,
 49: 0,
 50: 1,
 51: 0,
 52: 0,
 53: 0,
 54: 1,
 55: 0,
 56: 10,
 57: 1,
 58: 0,
 59: 3,
 60: 1,
 61: 7,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 5,
 67: 0,
 68: 0,
 69: 1,
 70: 3,
 71: 1,
 72: 4,
 73: 0,
 74: 2,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 5,
 80: 0,
 81: 1,
 82: 0,
 83: 1,
 84: 1,
 85: 0,
 86: 0,
 87: 1,
 88: 11,
 89: 0,
 90: 1,
 91: 12,
 92: 6,
 93: 0,
 94: 4,
 95: 13,
 96: 5,
 97: 7,
 98: 1,
 99: 1,
 100: 1,
 101: 3,
 102: 1,
 103: 2,
 104: 2,
 105: 14,
 106: 4,
 107: 0,
 108: 1,
 109: 0,
 110: 5,
 111: 4,
 112: 1,
 113: 1,
 114: 2,
 115: 5,
 116: 3,
 117: 0,
 118: 5,
 119: 0,
 120: 5,
 121: 13,
 122: 1,
 123

In [18]:
a = train[train['fiberID'] == 1000]
a

,type,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,...,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,type_num
id,,,,,,,,,,,,,,,,,,,,,
107,QSO,1000,21.431328,21.312061,20.611435,20.829270,20.845099,21.780833,21.645825,20.487463,...,21.112043,21.232596,21.031775,21.060400,21.386966,21.299095,21.121429,20.876343,20.798990,8
5596,QSO,1000,21.896253,21.966369,21.698764,21.390531,21.576770,22.200748,22.439993,21.991147,...,22.098364,22.361933,21.660624,20.839548,21.737560,22.004073,21.679964,21.311060,21.832921,8
9025,QSO,1000,22.730775,21.877607,21.648119,21.426117,21.004286,22.912756,22.146020,22.055225,...,21.825405,21.871610,21.675102,20.741863,23.713030,21.830070,21.636060,21.557654,21.278902,8
9055,QSO,1000,22.915844,20.635628,20.256953,20.190934,20.178427,22.621334,21.059316,20.662324,...,20.678344,20.361232,20.559165,20.364674,23.217457,20.648823,20.156205,20.259911,20.200615,8
15853,QSO,1000,21.774443,21.320292,21.081873,21.335658,20.882414,22.072457,21.338775,21.487418,...,21.494130,21.358868,21.304548,19.891201,21.807878,21.290518,21.044357,21.440006,20.332719,8
22280,QSO,1000,21.425084,20.669601,20.668940,20.462687,20.178813,21.421523,20.970182,21.058172,...,20.753148,20.671862,20.506668,20.273604,21.412894,20.664675,20.668003,20.449441,20.125002,8
22980,QSO,1000,22.138955,21.324781,21.160261,21.574242,21.792493,22.579844,21.542182,21.590913,...,21.358963,21.509293,21.958292,21.648084,22.245935,21.278329,21.150006,21.493579,21.435466,8
23914,QSO,1000,22.804563,21.729616,21.081705,20.596293,19.881660,22.516139,22.199345,21.296191,...,22.047780,21.104095,20.662945,20.133279,22.819930,21.781993,21.108836,20.432116,19.908645,8
24208,QSO,1000,21.785314,21.087300,20.945944,20.794163,20.711325,22.118710,21.478678,21.223733,...,21.189166,21.000868,20.922808,20.688900,21.870204,21.148168,20.843660,20.787475,20.313154,8


In [ ]:
'''
set(집합)
- 순서가 없고, 집합 안에서는 유니크한 값을 가짐
- 중괄호를 사용하는 것은 딕셔너리와 비슷하지만, key가 없음. 값만 존재
'''


In [7]:
ntrain = train.shape[0]  # 199991
ntest = test.shape[0]    # 10009
all_data = train.append(test)

In [8]:
all_data['fiberType'] = all_data['fiberID']
Fill_Type = lambda x: fiber_dict[x]
all_data['fiberType'] = all_data['fiberType'].apply(Fill_Type)

In [38]:
all_data.head()

,fiberID,fiberMag_g,fiberMag_i,fiberMag_r,fiberMag_u,fiberMag_z,modelMag_g,modelMag_i,modelMag_r,modelMag_u,...,petroMag_u,petroMag_z,psfMag_g,psfMag_i,psfMag_r,psfMag_u,psfMag_z,type,type_num,fiberType
id,,,,,,,,,,,,,,,,,,,,,
0,601,21.644453,21.387653,21.657571,22.581309,21.572827,21.465534,21.020605,21.364187,22.749241,...,22.504317,20.422446,21.431953,21.176553,21.314148,23.198224,21.171444,QSO,8.0,1
1,788,21.029773,20.937731,20.967054,21.868797,21.063646,20.758527,20.693389,20.753925,21.492955,...,21.360701,20.646660,20.708104,20.703420,20.678850,21.431355,20.473229,QSO,8.0,34
2,427,17.033098,17.095999,16.999682,18.171890,17.076449,16.697434,16.660177,16.641249,17.818063,...,17.867253,16.808006,16.727898,16.694640,16.679677,17.851451,16.641788,QSO,8.0,5
3,864,20.317165,20.073852,20.217898,21.039030,19.794505,20.001699,19.758113,19.889798,20.770711,...,20.433907,19.455117,20.040371,19.843840,19.926909,20.789900,19.463270,QSO,8.0,34
4,612,23.629122,19.861718,21.742750,25.700632,18.810375,23.147993,19.487330,21.475342,24.877052,...,25.859229,18.376141,23.058767,19.504961,21.471406,26.454969,18.389096,STAR_RED_DWARF,10.0,1


In [39]:
all_data.tail()

,fiberID,fiberMag_g,fiberMag_i,fiberMag_r,fiberMag_u,fiberMag_z,modelMag_g,modelMag_i,modelMag_r,modelMag_u,...,petroMag_u,petroMag_z,psfMag_g,psfMag_i,psfMag_r,psfMag_u,psfMag_z,type,type_num,fiberType
id,,,,,,,,,,,,,,,,,,,,,
209995,389,19.827172,19.686861,19.714033,20.093308,19.809697,19.536518,19.376445,19.432463,19.792181,...,19.882728,19.483389,19.542406,19.357957,19.439286,19.765035,19.357305,NaN,NaN,5
209996,247,19.928523,19.763771,19.696262,20.319914,19.407112,19.604800,19.401240,19.334113,20.066552,...,20.085454,19.390865,19.609379,19.391290,19.325949,19.960030,19.258862,NaN,NaN,5
209997,941,20.551190,20.200872,20.365800,21.254454,19.930425,20.255485,19.906014,20.141549,21.045501,...,20.998020,19.988363,20.276182,19.927208,20.090775,21.221899,19.766732,NaN,NaN,34
209998,674,21.950139,21.501469,21.610805,22.042942,21.641811,21.589489,21.430526,21.128330,21.700769,...,22.320173,21.331529,21.624585,21.191775,21.093379,21.625469,21.186596,NaN,NaN,34
209999,358,16.836321,16.373316,16.501623,17.747088,16.335839,16.530546,16.045822,16.176274,17.444318,...,17.481303,16.052459,16.531405,16.034888,16.186019,17.438361,15.995932,NaN,NaN,0


측정 별 변수

In [9]:
psfMags = ['psfMag_u','psfMag_g','psfMag_r','psfMag_i','psfMag_z']
fiberMags = ['fiberMag_u','fiberMag_g','fiberMag_r','fiberMag_i','fiberMag_z']
petroMags = ['petroMag_u','petroMag_g','petroMag_r','petroMag_i','petroMag_z']
modelMags = ['modelMag_u','modelMag_g','modelMag_r','modelMag_i','modelMag_z']

파장 별 변수

In [10]:
u_values = ['psfMag_u','fiberMag_u','petroMag_u','modelMag_u']
g_values = ['psfMag_g','fiberMag_g','petroMag_g','modelMag_g']
r_values = ['psfMag_r','fiberMag_r','petroMag_r','modelMag_r']
i_values = ['psfMag_i','fiberMag_i','petroMag_i','modelMag_i']
z_values = ['psfMag_z','fiberMag_z','petroMag_z','modelMag_z']

파장 변수 Engineering

강경수님의 토론 글을 읽고 구글링을 통해 u-r,u-g,g-r,g-z,g-i 등의 변수가 천체 분류에 효과적이라는 사실을 알아냄 50위권>10위권 상승

관련 자료

http://dspace.ewha.ac.kr/handle/2015.oak/211793

http://www.koreascience.or.kr/article/CFKO201023859828644.pub

*** 기타 변수들은 임의로 만들어봄

In [11]:
u_r = ['psfMag_u-r', 'fiberMag_u-r','petroMag_u-r','modelMag_u-r']
u_g = ['psfMag_u-g','fiberMag_u-g',  'petroMag_u-g','modelMag_u-g']
g_r = ['psfMag_g-r', 'fiberMag_g-r',  'petroMag_g-r', 'modelMag_g-r']      
g_z = ['psfMag_g-z','fiberMag_g-z', 'petroMag_g-z','modelMag_g-z'] 
g_i = ['psfMag_g-i','fiberMag_g-i','petroMag_g-i', 'modelMag_g-i']

u_i = ['psfMag_u-i','fiberMag_u-i','petroMag_u-i', 'modelMag_u-i']
u_z = ['psfMag_u-z','fiberMag_u-z','petroMag_u-z', 'modelMag_u-z']
r_i = ['psfMag_r-i','fiberMag_r-i','petroMag_r-i', 'modelMag_r-i']
r_z = ['psfMag_r-z','fiberMag_r-z','petroMag_r-z', 'modelMag_r-z']
i_z = ['psfMag_i-z','fiberMag_i-z','petroMag_i-z', 'modelMag_i-z']

In [12]:
sumur = ['psfMag_u+r', 'fiberMag_u+r','petroMag_u+r','modelMag_u+r']
sumug = ['psfMag_u+g','fiberMag_u+g',  'petroMag_u+g','modelMag_u+g']
sumgr = ['psfMag_g+r', 'fiberMag_g+r',  'petroMag_g+r', 'modelMag_g+r']      
sumgz = ['psfMag_g+z','fiberMag_g+z', 'petroMag_g+z','modelMag_g+z'] 
sumgi = ['psfMag_g+i','fiberMag_g+i','petroMag_g+i', 'modelMag_g+i']

sumui = ['psfMag_u+i','fiberMag_u+i','petroMag_u+i', 'modelMag_u+i']
sumuz = ['psfMag_u+z','fiberMag_u+z','petroMag_u+z', 'modelMag_u+z']
sumri = ['psfMag_r+i','fiberMag_r+i','petroMag_r+i', 'modelMag_r+i']
sumrz = ['psfMag_r+z','fiberMag_r+z','petroMag_r+z', 'modelMag_r+z']
sumiz = ['psfMag_i+z','fiberMag_i+z','petroMag_i+z', 'modelMag_i+z']

In [13]:
cols = all_data.columns.drop(['type','type_num','fiberType','fiberID'])
pca = PCA(n_components = 2)

x = all_data[cols].values
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['all_pca1','all_pca2'])
all_data = pd.concat([all_data,principalDf], axis =1)

In [14]:
cols

Index(['fiberMag_g', 'fiberMag_i', 'fiberMag_r', 'fiberMag_u', 'fiberMag_z',
       'modelMag_g', 'modelMag_i', 'modelMag_r', 'modelMag_u', 'modelMag_z',
       'petroMag_g', 'petroMag_i', 'petroMag_r', 'petroMag_u', 'petroMag_z',
       'psfMag_g', 'psfMag_i', 'psfMag_r', 'psfMag_u', 'psfMag_z'],
      dtype='object')

In [55]:
all_data.head()

,fiberID,fiberMag_g,fiberMag_i,fiberMag_r,fiberMag_u,fiberMag_z,modelMag_g,modelMag_i,modelMag_r,modelMag_u,...,psfMag_g,psfMag_i,psfMag_r,psfMag_u,psfMag_z,type,type_num,fiberType,all_pca1,all_pca2
0,601,21.644453,21.387653,21.657571,22.581309,21.572827,21.465534,21.020605,21.364187,22.749241,...,21.431953,21.176553,21.314148,23.198224,21.171444,QSO,8.0,1,-30.656018,3.954571
1,788,21.029773,20.937731,20.967054,21.868797,21.063646,20.758527,20.693389,20.753925,21.492955,...,20.708104,20.703420,20.678850,21.431355,20.473229,QSO,8.0,34,-28.815317,3.027193
2,427,17.033098,17.095999,16.999682,18.171890,17.076449,16.697434,16.660177,16.641249,17.818063,...,16.727898,16.694640,16.679677,17.851451,16.641788,QSO,8.0,5,-24.398853,-2.852857
3,864,20.317165,20.073852,20.217898,21.039030,19.794505,20.001699,19.758113,19.889798,20.770711,...,20.040371,19.843840,19.926909,20.789900,19.463270,QSO,8.0,34,-27.984306,1.886524
4,612,23.629122,19.861718,21.742750,25.700632,18.810375,23.147993,19.487330,21.475342,24.877052,...,23.058767,19.504961,21.471406,26.454969,18.389096,STAR_RED_DWARF,10.0,1,-34.578423,5.922756


In [15]:
for i in range(4):
    all_data[str(u_values[i])+'-'+'r'] = all_data[u_values[i]]-all_data[r_values[i]]
    all_data[str(u_values[i])+'-'+'g'] = all_data[u_values[i]]-all_data[g_values[i]]
    all_data[str(g_values[i])+'-'+'r'] = all_data[g_values[i]]-all_data[r_values[i]]
    all_data[str(g_values[i])+'-'+'z'] = all_data[g_values[i]]-all_data[z_values[i]]
    all_data[str(g_values[i])+'-'+'i'] = all_data[g_values[i]]-all_data[i_values[i]]
    
    all_data[str(u_values[i])+'-'+'i'] = all_data[u_values[i]]-all_data[i_values[i]]
    all_data[str(u_values[i])+'-'+'z'] = all_data[u_values[i]]-all_data[z_values[i]]
    all_data[str(r_values[i])+'-'+'z'] = all_data[r_values[i]]-all_data[z_values[i]]
    all_data[str(r_values[i])+'-'+'i'] = all_data[r_values[i]]-all_data[i_values[i]]
    all_data[str(i_values[i])+'-'+'z'] = all_data[i_values[i]]-all_data[z_values[i]]
        
        
        
    all_data[str(u_values[i])+'+'+'r'] = all_data[u_values[i]]+all_data[r_values[i]]
    all_data[str(u_values[i])+'+'+'g'] = all_data[u_values[i]]+all_data[g_values[i]]
    all_data[str(g_values[i])+'+'+'r'] = all_data[g_values[i]]+all_data[r_values[i]]
    all_data[str(g_values[i])+'+'+'z'] = all_data[g_values[i]]+all_data[z_values[i]]
    all_data[str(g_values[i])+'+'+'i'] = all_data[g_values[i]]+all_data[i_values[i]]
    
    all_data[str(u_values[i])+'+'+'i'] = all_data[u_values[i]]+all_data[i_values[i]]
    all_data[str(u_values[i])+'+'+'z'] = all_data[u_values[i]]+all_data[z_values[i]]
    all_data[str(r_values[i])+'+'+'z'] = all_data[r_values[i]]+all_data[z_values[i]]
    all_data[str(r_values[i])+'+'+'i'] = all_data[r_values[i]]+all_data[i_values[i]]
    all_data[str(i_values[i])+'+'+'z'] = all_data[i_values[i]]+all_data[z_values[i]]


In [16]:
val_groups = [psfMags,fiberMags,petroMags,modelMags,u_values,g_values,r_values,i_values,z_values,u_r,u_g,g_r,g_z,g_i,u_i,u_z,r_i,r_z,i_z,sumur, sumug, sumgr,  sumgz, sumgi, sumui, sumuz, sumri ,sumrz ,sumiz]
group_names = ['psf','fiber','petro','model','u','g','r','i','z','u-r','u-g','g-r','g-z','g-i','u-i','u-z','r-i','r-z','i-z','sumur', 'sumug', 'sumgr',  'sumgz', 'sumgi', 'sumui', 'sumuz', 'sumri' ,'sumrz' ,'sumiz']

In [17]:
pca = PCA(n_components = 3)

for i in range(len(val_groups)):
    
    all_data[str(group_names[i])+'_mean'] = all_data[val_groups[i]].mean(axis =1)
    all_data[str(group_names[i])+'_std'] = all_data[val_groups[i]].std(axis =1)

    x = all_data[val_groups[i]].values
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = [str(group_names[i]) + '_pca1', str(group_names[i]) + '_pca2',str(group_names[i]) + '_pca3'])
    #print(group_names[i])
    #print('explained variance ratio :', pca.explained_variance_ratio_)

    
    
    all_data = pd.concat([all_data,principalDf],axis =1)

In [18]:
all_data.head()

,fiberID,fiberMag_g,fiberMag_i,fiberMag_r,fiberMag_u,fiberMag_z,modelMag_g,modelMag_i,modelMag_r,modelMag_u,...,sumrz_mean,sumrz_std,sumrz_pca1,sumrz_pca2,sumrz_pca3,sumiz_mean,sumiz_std,sumiz_pca1,sumiz_pca2,sumiz_pca3
0,601,21.644453,21.387653,21.657571,22.581309,21.572827,21.465534,21.020605,21.364187,22.749241,...,42.532069,0.544304,-12.512414,-0.534852,-0.697637,42.261069,0.573280,-13.291764,0.298846,-0.039251
1,788,21.029773,20.937731,20.967054,21.868797,21.063646,20.758527,20.693389,20.753925,21.492955,...,41.496346,0.390984,-10.401651,0.031635,-0.513244,41.417550,0.391981,-11.587265,-0.115271,0.355615
2,427,17.033098,17.095999,16.999682,18.171890,17.076449,16.697434,16.660177,16.641249,17.818063,...,33.556163,0.355918,5.256082,-0.014131,1.349037,33.602549,0.392586,4.034574,0.035688,-0.102864
3,864,20.317165,20.073852,20.217898,21.039030,19.794505,20.001699,19.758113,19.889798,20.770711,...,39.571471,0.294956,-6.642292,-0.032060,0.056146,39.423117,0.300776,-7.608096,-0.103076,-0.002439
4,612,23.629122,19.861718,21.742750,25.700632,18.810375,23.147993,19.487330,21.475342,24.877052,...,40.078579,0.329332,-7.622481,0.045218,-0.159800,38.103822,0.382441,-4.963862,-0.050720,0.130991


트리 구조 특성상 더 잘 학습시키기 위해 45 Rotation한 변수 생성

***행렬 곱을 이용해 더 효과적인 코드를 짰지만 무슨 이유인지 성능이 좋지 않아 이 방법을 씀

In [19]:
def RotateXY(x,y,xc=0,yc=0,angle=0,units="DEGREES"):  
    """Rotate an xy cooordinate about a specified origin  
  
    x,y      xy coordinates  
    xc,yc   center of rotation  
    angle   angle  
    units    "DEGREES" (default) or "RADIANS"  
    """  
    import math  
    x = x - xc  
    y = y - yc  
    if units == "DEGREES":  
        angle = math.radians(angle)  
    xr = (x * math.cos(angle)) - (y * math.sin(angle)) + xc  
    yr = (x * math.sin(angle)) + (y * math.cos(angle)) + yc  
    return xr, yr  

In [20]:
import datetime
for i in range(4):
    start = datetime.datetime.now()
    u_g_1 = []
    u_g_2 = []
    g_r_1 = []
    g_r_2 = []
    g_z_1 = []
    g_z_2 = []
    g_i_1 = []
    g_i_2 = []
    r_i_1 = []
    r_i_2 = []
        
    for k in range(len(all_data)):
        
        Rotate = RotateXY(all_data[str(u_values[i])+'-'+'g'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        u_g_1.append(Rotate[0])
        u_g_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'r'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_r_1.append(Rotate[0])
        g_r_2.append(Rotate[1])
        
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'z'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_z_1.append(Rotate[0])
        g_z_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'i'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_i_1.append(Rotate[0])
        g_i_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(r_values[i])+'-'+'i'] .loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        r_i_1.append(Rotate[0])
        r_i_2.append(Rotate[1])   
        
    ug1= pd.DataFrame(data = u_g_1
             , columns = [str(u_values[i])+'-'+'g'+'r451'])
    ug2= pd.DataFrame(data = u_g_2
             , columns = [str(u_values[i])+'-'+'g'+'r452'])
       
    gr1= pd.DataFrame(data = g_r_1
             , columns = [str(g_values[i])+'-'+'r'+'r451'])
    gr2= pd.DataFrame(data = g_r_2
             , columns = [str(g_values[i])+'-'+'r'+'r452'])
        
    gz1= pd.DataFrame(data = g_z_1
             , columns = [str(g_values[i])+'-'+'z'+'r451'])
    gz2= pd.DataFrame(data = g_z_2
             , columns = [str(g_values[i])+'-'+'z'+'r452'])
        
        
    gi1= pd.DataFrame(data = g_i_1
             , columns = [str(g_values[i])+'-'+'i'+'r451'])
    gi2= pd.DataFrame(data = g_i_2
             , columns = [str(g_values[i])+'-'+'i'+'r452'])   
    
    ri1= pd.DataFrame(data = r_i_1
             , columns = [str(r_values[i])+'-'+'i'+'r451'])
    ri2= pd.DataFrame(data = r_i_2
             , columns = [str(r_values[i])+'-'+'i'+'r452'])
             
    
    all_data = pd.concat([all_data,ug1,ug2,gr1,gr2,gz1,gz2,gi1,gi1,ri1,ri2],axis =1)
    runtime = datetime.datetime.now() - start
    break    

In [21]:
train= all_data[:ntrain]
test= all_data[ntrain:]

Train set에서 FiberType ==34인것 제외

output이 모두 QSO인 FiberID들은 train에서 제외하고 추후에 QSO 확률 1,나머지 0으로 채워넣음

In [22]:
y = train['type_num']
train = train.drop(columns=['type','type_num'], axis=1)
train1 = train[train['fiberType'] != 34]
train2 = train[train['fiberType'] == 34]
train1_idx = train1.index 
train2_idx = train2.index
y1 = y.loc[train1_idx]

test = test.drop(['type','type_num'], axis=1)
test1 = test[test['fiberType'] != 34]
test2 = test[test['fiberType'] == 34]
test1_idx = test1.index
test2_idx = test2.index


학습 및 예측 (cv = 20)

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold = 20
kf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state = 42)
preds =[]
for train_index, test_index in kf.split(train1, y1):
    x_train, x_val = train1.iloc[train_index], train1.iloc[test_index]
    y_train, y_val = y1.iloc[train_index], y1.iloc[test_index]
    
    clf = lgb.LGBMClassifier(n_estimators = 10000, max_depth=10, learning_rate=0.009, objective='multiclass',num_class = 19
                             ,num_leaves = 1024,random_state=42,metric = 'multi_logloss',boost_from_average = False,
                            feature_fraction =0.21815809919426804,bagging_fraction = 0.8055711236002633,lambda_l1 = 0.0029896841778409566,
                             lambda_l2 = 0.06905300928232105,min_split_gain = 0.05931126989247223,min_child_weight = 10)
    clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_val, y_val)],
            early_stopping_rounds=50, verbose=100)
    
'''
과적합을 방지하기 위해 num_leaves는 2^(max_depth)보다 작아야 한다.

'''

'''
eval_set ?
evaluation 세트, 즉 검증 세트를 지정하는 것
fit() 수행 시 학습하면서 반복적으로 예측 오류값을 줄일 수 있도록 학습이 진행되는데
이때 학습은 학습 데이터로 하되, 예측 오류값 평가는 eval_set로 지정된 검증 세트로 평가하는 방식

eval_set=[(X_train, y_train), (X_test, y_test)] 
학습은 (X_train, y_train), 검증은 (X_test, y_test) 데이터 세트를 이용하라는 것

'''

'''
다중 클래스 분류(multiclass) => softmax
https://wikidocs.net/35476
'''
    
    # fiberID ==34 인 test_set에 QSO =1로 채워넣음
    ps = pd.DataFrame(data=clf.predict_proba(test), index=test.index)
    sam = sample_submission.copy()
    sam.loc[test1_idx] =ps.loc[test1_idx]
    sam.loc[test2_idx] = 0
    sam.loc[test2_idx,'QSO'] = 1
    

    preds.append(sam)

In [ ]:
pred = sum(preds)/20
pred.index = sample_submission.index
pred.columns = sample_submission.columns
pred.to_csv('LGBM_20_Final.csv', index=True)

Hyperopt library를 이용한 하이퍼 파라미터 튜닝

https://www.kaggle.com/eikedehling/tune-and-compare-xgb-lightgbm-rf-with-hyperopt

https://www.kaggle.com/ilialar/hyperparameters-tunning-with-hyperopt